### 범주형 데이터 이진분류 경진대회 성능 개선 Ⅰ

In [1]:
import pandas as pd
import numpy as np
import os
import warnings

warnings.filterwarnings('ignore')

### 데이터 불러오기

In [2]:
cd ..

C:\Users\samsung\Kaggle


In [3]:
PATH = './cat-in-the-dat/'

In [4]:
train = pd.read_csv(os.path.join(PATH,'train.csv'), index_col='id')
test = pd.read_csv(os.path.join(PATH,'test.csv'), index_col='id')
submission = pd.read_csv(os.path.join(PATH,'sample_submission.csv'), index_col='id')

### Feature Engineering 1 : 피처 맞춤 인코딩

#### 데이터 합치기

In [5]:
all_data = pd.concat([train,test])
all_data = all_data.drop('target',axis=1)

#### 이진 피처 인코딩

In [6]:
all_data.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8


In [7]:
all_data['bin_3'] = all_data['bin_3'].map({'F':0, 'T':1})
all_data['bin_4'] = all_data['bin_4'].map({'N':0, 'Y':1})

#### 순서형 피처 인코딩

In [8]:
ord1dict = {'Novice':0, 'Contributor':1,
           'Expert':2, 'Master':3, 'Grandmaster':4}
ord2dict = {'Freezing':0, 'Cold':1, 'Warm':2,
           'Hot':3, 'Boiling Hot':4, 'Lava Hot':5}
all_data['ord_1'] = all_data['ord_1'].map(ord1dict)
all_data['ord_2'] = all_data['ord_2'].map(ord2dict)

In [9]:
from sklearn.preprocessing import OrdinalEncoder

ord_345 = ['ord_3','ord_4','ord_5']

ord_encoder = OrdinalEncoder() # OrdinalEncoder 객체 생성
all_data[ord_345] = ord_encoder.fit_tra- nsform(all_data[ord_345])

# 피처별 인코딩 순서 출력
for feature, categories in zip(ord_345, ord_encoder.categories_):
    print(feature)
    print(categories)

ord_3
['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o']
ord_4
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']
ord_5
['AP' 'Ai' 'Aj' 'BA' 'BE' 'Bb' 'Bd' 'Bn' 'CL' 'CM' 'CU' 'CZ' 'Cl' 'DH'
 'DN' 'Dc' 'Dx' 'Ed' 'Eg' 'Er' 'FI' 'Fd' 'Fo' 'GD' 'GJ' 'Gb' 'Gx' 'Hj'
 'IK' 'Id' 'JX' 'Jc' 'Jf' 'Jt' 'KR' 'KZ' 'Kf' 'Kq' 'LE' 'MC' 'MO' 'MV'
 'Mf' 'Ml' 'Mx' 'NV' 'Nf' 'Nk' 'OR' 'Ob' 'Os' 'PA' 'PQ' 'PZ' 'Ps' 'QM'
 'Qb' 'Qh' 'Qo' 'RG' 'RL' 'RP' 'Rm' 'Ry' 'SB' 'Sc' 'TR' 'TZ' 'To' 'UO'
 'Uk' 'Uu' 'Vf' 'Vx' 'WE' 'Wc' 'Wv' 'XI' 'Xh' 'Xi' 'YC' 'Yb' 'Ye' 'ZR'
 'ZS' 'Zc' 'Zq' 'aF' 'aM' 'aO' 'aP' 'ac' 'av' 'bF' 'bJ' 'be' 'cA' 'cG'
 'cW' 'ck' 'cp' 'dB' 'dE' 'dN' 'dO' 'dP' 'dQ' 'dZ' 'dh' 'eG' 'eQ' 'eb'
 'eg' 'ek' 'ex' 'fO' 'fh' 'gJ' 'gM' 'hL' 'hT' 'hh' 'hp' 'iT' 'ih' 'jS'
 'jV' 'je' 'jp' 'kC' 'kE' 'kK' 'kL' 'kU' 'kW' 'ke' 'kr' 'kw' 'lF' 'lL'
 'll' 'lx' 'mb' 'mc' 'mm' 'nX' 'nh' 'oC' 'oG' 'oH' 'oK' 'od' 'on' 'pa'
 'ps' 'qA' 'qJ' 'qK' 'qP' 'qX' '

#### 명목형 피처 인코딩

In [10]:
nom_features = ['nom_'+str(i) for i in range(10)]

In [11]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()
# 원-핫  인코딩 적용
encoded_nom_matrix = onehot_encoder.fit_transform(all_data[nom_features])

encoded_nom_matrix

<500000x16276 sparse matrix of type '<class 'numpy.float64'>'
	with 5000000 stored elements in Compressed Sparse Row format>

In [12]:
all_data = all_data.drop(nom_features, axis=1) # 기존 명목형 피처 삭제

#### 날짜 피처 인코딩

In [13]:
date_features = ['day','month']

# 원-핫  인코딩 적용
encoded_date_matrix = onehot_encoder.fit_transform(all_data[date_features])

encoded_date_matrix

<500000x19 sparse matrix of type '<class 'numpy.float64'>'
	with 1000000 stored elements in Compressed Sparse Row format>

### Feature Engineering 2 : Feature Scaling

#### 순서형 피처 스케일링

In [14]:
from sklearn.preprocessing import MinMaxScaler

ord_features = ['ord_' + str(i) for i in range(6)]
# min-max 정규화
all_data[ord_features] = MinMaxScaler().fit_transform(all_data[ord_features])

In [15]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 13 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   bin_0   500000 non-null  int64  
 1   bin_1   500000 non-null  int64  
 2   bin_2   500000 non-null  int64  
 3   bin_3   500000 non-null  int64  
 4   bin_4   500000 non-null  int64  
 5   ord_0   500000 non-null  float64
 6   ord_1   500000 non-null  float64
 7   ord_2   500000 non-null  float64
 8   ord_3   500000 non-null  float64
 9   ord_4   500000 non-null  float64
 10  ord_5   500000 non-null  float64
 11  day     500000 non-null  int64  
 12  month   500000 non-null  int64  
dtypes: float64(6), int64(7)
memory usage: 53.4 MB


#### 인코딩 및 스케일링된 피처 합치기

In [16]:
from scipy import sparse

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data),
                              encoded_nom_matrix,
                              encoded_date_matrix],
                             format='csr')

In [17]:
all_data_sprs

<500000x16308 sparse matrix of type '<class 'numpy.float64'>'
	with 10163718 stored elements in Compressed Sparse Row format>

### 데이터 나누기

In [18]:
num_train = len(train)

X_train = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target']

In [19]:
from sklearn.model_selection import train_test_split

X_train,X_valid, y_train, y_valid = train_test_split(X_train, y, test_size=0.1,
                                                    stratify = y, random_state=10)

### 하이퍼 파라미터 최적화

In [20]:
%%time

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 모델 생성
logistic_model = LogisticRegression()

# 하이퍼파라미터 값 목록
lr_params = {'C':[0.1, 0.125, 0.2], 'max_iter':[800,900,1000],
            'solver':['liblinear'], 'random_state':[42]}

# 그리드서치 객체 생성
gridsearch_logistic_model = GridSearchCV(estimator = logistic_model,
                                        param_grid = lr_params,
                                        scoring = 'roc_auc',
                                        cv = 5)
# 그리드서치 수행
gridsearch_logistic_model.fit(X_train, y_train)

print('최적 하이퍼파라미터 : ', gridsearch_logistic_model.best_params_)

최적 하이퍼파라미터 :  {'C': 0.125, 'max_iter': 800, 'random_state': 42, 'solver': 'liblinear'}
Wall time: 9min 28s


### 모델 성능 검증

In [22]:
y_valid_preds = gridsearch_logistic_model.predict_proba(X_valid)[:, 1]

In [23]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f'검증 데이터 ROC AUC : {roc_auc:.4f}')

검증 데이터 ROC AUC : 0.8045


### 예측 및 결과 제출

In [25]:
y_preds = gridsearch_logistic_model.predict_proba(X_test)[:,1]

submission['target'] = y_preds

In [26]:
import datetime
timestring = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
filename=f'cat-in-the-dat/submission-{timestring}.csv'

In [27]:
submission.to_csv(filename)